In [1]:
# !pip install --upgrade -q gin git+https://github.com/google/trax.git@v1.4.1

In [2]:
# bimport trax 
# from trax import layers as tl
# import os 
# import numpy as np
# import pandas as pd


# from utils import get_params, get_vocab
# import random as rnd

# # set random seeds to make this notebook easier to replicate
# trax.supervised.trainer_lib.init_random_number_generators(33)

In [ ]:
# %% libraries
import numpy as np
import pandas as pd
import os
import sys
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import torch

import matplotlib.pyplot as plt
%matplotlib inline

In [63]:
!pip install -U spacy

In [4]:
# %% Global Variables
datasetPath = "./TM-3-2020"
sourceCodePath = './src'

sys.path.append(os.path.join(os.getcwd(), sourceCodePath))

In [5]:
# %% Loads Dataset
import chat as ch

dataset = ch.dataset.Dataset(datasetPath)
# %%
df = dataset.get_dataframe()
df.head()

,conversation_id,scenario,instructions,conversation,intent,success
0,dlg-bca5ce0a-056f-446e-be94-3ba77b32a84f,auto template 1 with theater name error,"scenario: in the conversation below, a custome...",hi....am buying a ticket tonight so we go and ...,buy,1
1,dlg-bd494e2c-36f6-4529-8e4d-d5c4d64388ae,auto template 1 with theater name error,"scenario: in the conversation below, a custome...",I am looking for tickets tonight at the AMC Mo...,buy,1
2,dlg-c9064676-75fe-4d0a-83c2-497e1f2115a6,auto template 1 with theater name error,"scenario: in the conversation below, a custome...",I need to get some tickets for a movie tonight...,buy,1
3,dlg-f7500bcf-472c-48c3-adfd-e4ec9f63bcf1,auto template 1 with theater name error,"scenario: in the conversation below, a custome...",I need help finding showtimes for tonight at m...,buy,1
4,dlg-df1f0d45-27f2-4fb0-8aaa-b6b5f5a843bb,auto template 1 with theater name error,"scenario: in the conversation below, a custome...","Hello, I am interested in buying tickets tonig...",buy,1


In [31]:
df['conversation'].head(1)[0]


1075

In [67]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.0/en_core_web_sm-3.4.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-10-19 14:59:10.458872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudnn64_8.dll'; dlerror: cudnn64_8.dll not found
2022-10-19 14:59:10.458902: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [68]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [80]:
# %% build entities
def detect_times(text):
  time_selector = "(\\d{1,2}([:;.]\\d{2})?[ap]m)"
  result = re.findall(time_selector, text)

  if len(result) > 0:
    for i, item in enumerate(result):
      text = text.replace(item, 'time_' + str(i), 1)

  return text


dic = {}
i = 0

for row in df.iterrows():
  if i > 1:
    break

  i += 1

  # --------------------------opt 2
  detections = nlp(row[1].conversation)
  spacy.displacy.render(detections, style='ent')

  #  ------------------------- opt 1
  # detect_times(row[1].conversation)
  # print('conv:', row[1].conversation)